In [ ]:
import re
import os

from tqdm import tqdm, tnrange
import json
import pandas as pd
import os
from collections import namedtuple


from nltk.corpus import wordnet as wn
import copy
import numpy as np

import tensorflow.keras as K

import utils
import parsers
import models

training_file_path = '../resources/WSD_Evaluation_Framework/Training_Corpora/SemCor/semcor.data.xml'
gold_file_path =  '../resources/WSD_Evaluation_Framework/Training_Corpora/SemCor/semcor.gold.key.txt'

# training_file_path = '../resources/WSD_Evaluation_Framework/Evaluation_Datasets/semeval2007/semeval2007.data.xml'
# gold_file_path = '../resources/WSD_Evaluation_Framework/Evaluation_Datasets/semeval2007/semeval2007.gold.key.txt'

training_file_path_dev = '../resources/WSD_Evaluation_Framework/Evaluation_Datasets/semeval2013/semeval2013.data.xml'
gold_file_path_dev = '../resources/WSD_Evaluation_Framework/Evaluation_Datasets/semeval2013/semeval2013.gold.key.txt'
fine_senses_vocab_path = '../resources/semcor.vocab.WordNet.json'
input_vocab_path = '../resources/semcor.input.vocab.json'
input_antivocab_path = '../resources/semcor.leftout.vocab.json'
embedding_size = 32
batch_size = 64
LEARNING_RATE = 0.01
N_EPOCHS = 10
PADDING_SIZE = 50
print_model = False

In [ ]:
#loading dict
senses = utils.json_vocab_reader(fine_senses_vocab_path)
inputs, antivocab = utils.json_vocab_reader(input_vocab_path, input_antivocab_path)
output_vocab = utils.vocab_merge(senses, inputs)
reverse_output_vocab =  dict((v, k) for k, v in output_vocab.items())

K.backend.clear_session()

In [ ]:
dir_ = "../resources/WSD_Evaluation_Framework/Evaluation_Datasets"
eval_datasets = [i for i in os.listdir(dir_) if i.startswith("se")]
eval_datasets

In [ ]:
# training_file_path = '../resources/WSD_Evaluation_Framework/Evaluation_Datasets/semeval2015/semeval2015.data.xml'


In [ ]:
# ##################
# # Model loading #
# #################
loaded_model = K.models.load_model('../resources/models/model_2019-09-06_22:48:47_+0200.h5')
loaded_model.load_weights('../resources/models/model_weights_2019-09-06_22:48:47_+0200.h5')


In [ ]:
# loaded_model.compile(loss = 'categorical_crossentropy',
#               optimizer = K.optimizers.Adam(lr=0.0015, clipnorm=1., clipvalue=0.5),
#               metrics = ['acc', K.metrics.Precision()])

# ## PREDICTION
# predicted_one_hot = loaded_model.predict([padded(X_test_uni), padded(X_test_bi)])
# #de - one hot encode
# predicted = np.argmax(predicted_one_hot, axis=2)

In [7]:
import generatorPrototype
import generators

In [156]:
def eval_parser(path = training_file_path_dev, batch_size = 64):
    
    data_flow = parsers.TrainingParser(path)
    sentence_batch = []
    for batch_count, sentence in enumerate(data_flow.parse(), start = 1):
        sentence_batch.append(sentence)

        if len(sentence_batch)==(batch_size):#==0:
            yield sentence_batch
            sentence_batch = []

    if len(sentence_batch)>0:
        yield sentence_batch


In [204]:
def basic_predict(batch_ground_truth_sentences, batch_model_predictions, candidate_synsets):
    
    outputs = []
    output = namedtuple("output", "Sentence_id WordNet")
    
    for idx_sentence, sentence in enumerate(batch_model_predictions):

        ground_truth_sentence = batch_ground_truth_sentences[idx_sentence]
        
        #split based on Model padding
        #mfs_part = ground_truth_sentence[PADDING_SIZE+1:]
        #ground_truth_sentence = ground_truth_sentence[:PADDING_SIZE]
        
        for idx, entry in enumerate(ground_truth_sentence):
            
            if entry.instance == True: #only for instances not wf
                if idx<PADDING_SIZE: 
                    #WSD argmax
                    word_prob = sentence[idx]
                    current_candidate_synsets = candidate_synsets[idx_sentence][idx]
                    prob_dist_candidate_synset = word_prob[current_candidate_synsets]
                    current_synset = np.argmax(prob_dist_candidate_synset)

                    if current_synset>4: #change after deleting start stop
                        item = output(Sentence_id = entry.id_, WordNet = reverse_output_vocab[current_synset])
                        outputs.append(item)
                    else: #fallback
                        word = entry.lemma
                        item = output(Sentence_id = entry.id_, WordNet = models.MFS.retrieve_item(word))
                        outputs.append(item)
                else:
                    word = entry.lemma
                    item = output(Sentence_id = entry.id_, WordNet = models.MFS.retrieve_item(word))
                    outputs.append(item)
                
    return outputs

In [205]:
#training_file_path_dev = '../resources/WSD_Evaluation_Framework/Evaluation_Datasets/semeval2007/semeval2007.data.xml'
training_file_path_dev = '../resources/WSD_Evaluation_Framework/Evaluation_Datasets/semeval2013/semeval2013.data.xml'


In [206]:
eval_generator = generatorPrototype.get(batch_size = batch_size,
                                training_file_path = training_file_path_dev,
                                antivocab = antivocab,
                                output_vocab = output_vocab,
                                PADDING_SIZE = PADDING_SIZE)
real_words = eval_parser(path = training_file_path_dev, batch_size = 64)

for batch_ground_truth_sentences in real_words:

    batch_x, candidate_synsets = next(eval_generator)

    batch_model_predictions = loaded_model.predict_on_batch(batch_x)
    
    outputs = basic_predict(batch_ground_truth_sentences, batch_model_predictions, candidate_synsets)
    print(len(outputs))
    
    #write to file

368
360
349
300
267


In [210]:
outputs

[output(Sentence_id='d011.s000.t000', WordNet='wn:00056311n'),
 output(Sentence_id='d011.s000.t001', WordNet='wn:05832745n'),
 output(Sentence_id='d011.s000.t002', WordNet='wn:14483917n'),
 output(Sentence_id='d011.s000.t003', WordNet='wn:08366753n'),
 output(Sentence_id='d011.s001.t000', WordNet='wn:00056311n'),
 output(Sentence_id='d011.s001.t001', WordNet='wn:05901508n'),
 output(Sentence_id='d011.s001.t002', WordNet='wn:05850624n'),
 output(Sentence_id='d011.s001.t003', WordNet='wn:14320394n'),
 output(Sentence_id='d011.s001.t004', WordNet='wn:08209687n'),
 output(Sentence_id='d011.s001.t005', WordNet='wn:02604760v'),
 output(Sentence_id='d011.s001.t006', WordNet='wn:05901508n'),
 output(Sentence_id='d011.s001.t007', WordNet='wn:02604760v'),
 output(Sentence_id='d011.s001.t008', WordNet='wn:07554758n'),
 output(Sentence_id='d011.s001.t009', WordNet='wn:05901508n'),
 output(Sentence_id='d011.s001.t010', WordNet='wn:13945919n'),
 output(Sentence_id='d011.s001.t011', WordNet='wn:08168

In [195]:
def basic_predict(batch_ground_truth_sentences, batch_model_predictions, candidate_synsets):
    
    outputs = []
    output = namedtuple("output", "Sentence_id WordNet")
    
    for idx_sentence, sentence in enumerate(batch_model_predictions):
        ground_truth_sentence = batch_ground_truth_sentences[idx_sentence]
        #print(len(ground_truth_sentence))
        #split based on Model padding
        mfs_part = ground_truth_sentence[PADDING_SIZE:]
        ground_truth_sentence = ground_truth_sentence[:PADDING_SIZE]
        #print(len(ground_truth_sentence))
        #print(len(ground_truth_sentence))
        #print("------------_")
        
        for idx, entry in enumerate(ground_truth_sentence):
           # print(idx_sentence, idx)
            if entry.instance == True: #only for instances not wf
                if idx<PADDING_SIZE: 
                    #WSD argmax
                    print(idx_sentence, idx)
                    word_prob = sentence[idx]
                    current_candidate_synsets = candidate_synsets[idx_sentence][idx]
                    prob_dist_candidate_synset = word_prob[current_candidate_synsets]
                    current_synset = np.argmax(prob_dist_candidate_synset)

                    if current_synset>4: #change after deleting start stop
                        item = output(Sentence_id = entry.id_, WordNet = reverse_output_vocab[current_synset])
                        outputs.append(item)
                    else: #fallback
                        word = entry.lemma
                        item = output(Sentence_id = entry.id_, WordNet = models.MFS.retrieve_item(word))
                        outputs.append(item)
                else:
                    print("sex on the beach")
                    word = entry.lemma
                    item = output(Sentence_id = entry.id_, WordNet = models.MFS.retrieve_item(word))
                    outputs.append(item)
                    
        for idx, entry in enumerate(mfs_part):
            word = entry.lemma
            item = output(Sentence_id = entry.id_, WordNet = models.MFS.retrieve_item(word))
            outputs.append(item)
                
    return outputs


In [197]:
#basic_predict(batch_ground_truth_sentences, batch_model_predictions, candidate_synsets)

In [201]:
len(batch_model_predictions)

64

In [189]:
x[3:]

[4]

In [152]:
real_words = utils.eval_parser(path = training_file_path_dev, batch_size = batch_size)

for batch_x, candidate_synsets in eval_generator:
    print(batch_x.shape)
#     batch_real_words = next(real_words)
#     batch_predictions = loaded_model.predict_on_batch(batch_x)
    
    #batch_ground_truth_sentences = next(real_words)
    #print(len(batch_ground_truth_sentences))
   # batch_model_predictions = loaded_model.predict_on_batch(batch_x)
    #print(len(batch_ground_truth_sentences))
    #outputs = basic_predict(batch_ground_truth_sentences, batch_model_predictions)
    #print(len(outputs))

In [75]:
len(utils.eval_parser(path = training_file_path_dev, batch_size = 64).__next__())


64

In [ ]:
# real_words = utils.eval_parser(path = training_file_path_dev, batch_size = 64)
# for i in real_words:
#     #print(len(i))
#     for q in i:
#         for word in q:
#             if word.id_ is not None:
#                 print(word.id_)

In [108]:
eval_generator = generatorPrototype.get(batch_size = batch_size,
                                training_file_path = training_file_path_dev,
                                antivocab = antivocab,
                                output_vocab = output_vocab,
                                PADDING_SIZE = PADDING_SIZE)

# real_words = utils.eval_parser(path = training_file_path_dev, batch_size = 64)
# for batch_real_words in real_words:
#     batch_x, candidate_synsets = next(eval_generator)
#     #print(batch_x.shape)
#     batch_predictions = loaded_model.predict_on_batch(batch_x)
#     outputs = basic_predict(batch_ground_truth_sentences, batch_model_predictions)
#     print(len(outputs))

In [109]:
real_words = utils.eval_parser(path = training_file_path_dev, batch_size = batch_size)

for batch_x, candidate_synsets in eval_generator:
    print(batch_x.shape)
#     batch_real_words = next(real_words)
#     batch_predictions = loaded_model.predict_on_batch(batch_x)
    
    #batch_ground_truth_sentences = next(real_words)
    #print(len(batch_ground_truth_sentences))
   # batch_model_predictions = loaded_model.predict_on_batch(batch_x)
    #print(len(batch_ground_truth_sentences))
    #outputs = basic_predict(batch_ground_truth_sentences, batch_model_predictions)
    #print(len(outputs))

ValueError: too many values to unpack (expected 2)

In [ ]:
next(eval_generator)[2]

In [ ]:
batch_size = 64

In [ ]:
batch_model_predictions.shape

In [ ]:
for batch_real_words in real_words:
    print(len(batch_real_words))

In [142]:
eval_generator = generatorPrototype.get(batch_size = batch_size,
                                training_file_path = training_file_path_dev,
                                antivocab = antivocab,
                                output_vocab = output_vocab,
                                PADDING_SIZE = PADDING_SIZE)

real_words = eval_parser(path = training_file_path_dev, batch_size = batch_size)

for batch_x, candidate_synsets in eval_generator:

    batch_real_words = next(real_words)
    batch_predictions = loaded_model.predict_on_batch(batch_x)
    
    batch_ground_truth_sentences = next(real_words)
    batch_model_predictions = loaded_model.predict_on_batch(batch_x)
    
    for idx_sentence, sentence in enumerate(batch_predictions):

        ground_truth_sentence = batch_ground_truth_sentences[idx_sentence]

        #split based on Model padding
        ground_truth_sentence = ground_truth_sentence[:PADDING_SIZE]
        mfs_part = ground_truth_sentence[PADDING_SIZE+1:]
        
        for idx, entry in enumerate(ground_truth_sentence):
            
            #only for instances not wf
            if entry.instance == True:
                if idx<PADDING_SIZE:
                    
                    #WSD argmax
                    word_prob = sentence[idx]
                    current_candidate_synsets = candidate_synsets[idx_sentence][idx]
                    prob_dist_candidate_synset = word_prob[current_candidate_synsets]
                    current_synset = np.argmax(prob_dist_candidate_synset)

                    if current_synset>4: #change after deleting start stop
                        single_lemma_prediction = reverse_output_vocab[current_synset]
                        print(entry.id_, single_lemma_prediction)
                    else:
                        word = entry.lemma
                        print(entry.id_, models.MFS.retrieve_item(word))
                        
                else:
                    word = entry.lemma
                    print(entry.id_, models.MFS.retrieve_item(word))

d003.s006.t000 wn:08058098n
d003.s006.t001 wn:01110274n
d003.s006.t002 wn:08569998n
d003.s006.t003 wn:15211806n
d003.s007.t000 wn:05926676n
d003.s007.t001 wn:14966667n
d003.s007.t002 wn:01323781n
d003.s007.t003 wn:08913434n
d003.s007.t004 wn:08069050n
d003.s007.t005 wn:09790482n
d003.s008.t000 wn:08913434n
d003.s008.t001 wn:03221720n
d003.s008.t002 wn:15203791n
d003.s008.t003 wn:00973077n
d003.s008.t004 wn:08355791n
d003.s008.t005 wn:08058098n
d003.s008.t006 wn:04928903n
d003.s008.t007 wn:10044879n
d003.s008.t008 wn:06642138n
d003.s008.t009 wn:03748162n
d003.s009.t000 wn:15242955n
d003.s009.t001 wn:07185076n
d003.s009.t002 wn:08913434n
d003.s010.t000 wn:05670710n
d003.s010.t001 wn:14539268n
d003.s010.t002 wn:00978413n
d003.s010.t003 wn:15164105n
d003.s010.t004 wn:13975659n
d003.s010.t005 wn:08355791n
d003.s010.t006 wn:08199025n
d003.s010.t007 wn:08069241n
d003.s010.t008 wn:08913434n
d003.s010.t009 wn:09466280n
d003.s010.t010 wn:04900121n
d003.s010.t011 wn:09790482n
d003.s011.t000 wn:08

StopIteration: 

In [ ]:
next(real_words)

In [ ]:
print(batch_model_predictions.shape)

In [ ]:
class MFS(object):
    """
    WSD performed via Most Frequent Sense (MFS) which always returns the predominant sense of a lemma
    """

    def retrieve_item(lemma):
        """
        Retreives the MFS of a lemma from the nltk package if exists else word
        """
        synsets = wn.synsets(str(lemma))
        
        if len(synsets) == 0:
            print("fail")
            return lemma
        else:
            mfs = synsets[0]
            return utils.WordNet.from_synset(mfs)

    def predict(sentence):
        """
        param: sentence of lemmas
        return sentence MFS
        """
        return [MFS.retrieve_item(lemma) for lemma in sentence]

In [ ]:
PADDING_SIZE = 50
for idx_sentence, sentence in enumerate(predictions):
    print(idx_sentence)
    real_sentence = batch_real_words[idx_sentence]
    
    #split based on Model padding
    real_sentence, mfs_part = real_sentence[:50], real_sentence[51:]
    for idx, entry in enumerate(real_sentence):
        
        if entry.instance == True:
            if idx<PADDING_SIZE:
                word_prob = sentence[idx]

                current_candidate_synsets = candidate_synsets[idx_sentence][idx]
                prob_dist_candidate_synset = word_prob[current_candidate_synsets]
                current_synset = np.argmax(prob_dist_candidate_synset)

                if current_synset>4:#change after deleting start stop
                    single_lemma_prediction = reverse_output_vocab[current_synset]
                    print(entry.id_, single_lemma_prediction)
                else:
                    word = entry.lemma
                    print(entry.id_, MFS.retrieve_item(word))
            else:
                word = entry.lemma
                print(entry.id_, MFS.retrieve_item(word))
